Tweet data is stored in JSON format by Twitter. Getting tweet JSON data via tweet ID using Tweepy is described well in this StackOverflow answer. Note that setting the tweet_mode parameter to 'extended' in the get_status call, i.e., api.get_status(tweet_id, tweet_mode='extended'), can be useful.

Also, note that the tweets corresponding to a few tweet IDs in the archive may have been deleted. Try-except blocks may come in handy here.

# Introduction and import of modules

This project aims to use the tweet archive of twitter user `@dog_rates` also known as `WeRateDogs` . The project focusses on collecting, assessing and cleaning the data collected through the Twitter API using `tweepy`. After Wrangling an analysis is made of the data. The initial estimate of necessary modules is as follows:
* pandas
* numpy
* matplotlib
* re
* statsmodels

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import statsmodels.api as sm
%matplotlib inline

In [8]:
import tweepy

consumer_key = 'aNTJRrnNf2PI9JXP7AvpkTB4M'
consumer_secret = 'bPu9Rcwuyn3ps8JNN6UqzEZYfmWo7RbboC8YcNaCLQ3NicD0Ci'
access_token = '92779782-ry7yZqmQq6PvQN9p1dF2pYDo9BBDInMybnwBidmlT'
access_secret = 'Rt5d2aEbsq1NXwDwFKnVBogSgmrRnZew4RnQKcFqNfXjm'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

# Description of data sources and import of data

Import of data:

WeRateDogs Twitter archive is available as a csv file, provided by Udacity. the filename is `twitter_archive_enhanced.csv`

The neural network generated file with predictions of dog breeds based on the images in the twitter archive is available from the following website https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv

Use the tweet ID's in the archive, to query the twitter API for additional data, write the additional data to `tweet_json.txt`. Gather at minimum
* tweet id
* Retweet count
* favorite count
Write each tweets json data to a new line in the `tweet_json.txt` file. use:https://stackabuse.com/reading-and-writing-json-to-a-file-in-python/

In [9]:
#load csv file
twitter_archive = pd.read_csv("twitter_archive_enhanced.csv")
#check if it worked
twitter_archive.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo;;
0,"892420643555336193,,,2017-08-01 16:23:56 +0000...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"892177421306343426,,,2017-08-01 00:17:27 +0000...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"891815181378084864,,,2017-07-31 00:18:03 +0000...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"891689557279858688,,,2017-07-30 15:58:51 +0000...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"891327558926688256,,,2017-07-29 16:00:24 +0000...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
twitter_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2517 entries, 0 to 2516
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2517 non-null   object 
 1   in_reply_to_status_id       8 non-null      object 
 2   in_reply_to_user_id         3 non-null      object 
 3   timestamp                   3 non-null      object 
 4   source                      45 non-null     object 
 5   text                        52 non-null     object 
 6   retweeted_status_id         52 non-null     object 
 7   retweeted_status_user_id    52 non-null     object 
 8   retweeted_status_timestamp  52 non-null     object 
 9   expanded_urls               52 non-null     object 
 10  rating_numerator            52 non-null     object 
 11  rating_denominator          52 non-null     object 
 12  name                        9 non-null      object 
 13  doggo                       7 non

In [11]:
twitter_archive.at[0, 'timestamp']

nan

In [12]:
#copied a tweet_id from the csv file for testing purposes with tweepy
test_id = '892420643555336193'
testtweet = api.get_status(test_id)
testtweet.retweet_count, testtweet.favorite_count

(7461, 35352)



        created_at : The time the status was posted.
        id : The ID of the status.
        id_str : The ID of the status as a string.
        text : The text of the status.
        entities : The parsed entities of the status such as hashtags, URLs etc.
        source : The source of the status.
        source_url : The URL of the source of the status.
        in_reply_to_status_id : The ID of the status being replied to.
        in_reply_to_status_id_str : The ID of the status being replied to in as a string.
        in_reply_to_user_id : The ID of the user being replied to.
        in_reply_to_user_id_str : The ID of the user being replied to as a string.
        in_reply_to_screen_name : The screen name of the user being replied to
        user : The User object of the poster of the status.
        geo : The geo object of the status.
        coordinates : The coordinates of the status.
        place : The place of the status.
        contributors : The contributors of the status.
        is_quote_status : Indicates whether the status is a quoted status or not.
        retweet_count : The number of retweets of the status.
        favorite_count : The number of likes of the status.
        favorited : Indicates whether the status has been favourited by the authenticated user or not.
        retweeted : Indicates whether the status has been retweeted by the authenticated user or not.
        possibly_sensitive : Indicates whether the status is sensitive or not.
        lang : The language of the status.



In [13]:
#load additional modules
import requests
import os
# download the image pridections file
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)
#check if response worked
response

<Response [200]>

In [14]:
#download dog breed predictions file and store in base directory
with open('predictions.tsv',mode = 'wb') as file:
    file.write(response.content)

In [15]:
#check if file is downloaded
os.listdir()

['.ipynb_checkpoints',
 'predictions.tsv',
 'Project_4_Wrangling_Twitter.ipynb',
 'Readme.md',
 'twitter_archive_enhanced.csv']



# Data Wrangling

Key points for data wrangling in the assignment
Key Points

Key points to keep in mind when data wrangling for this project:

 * You only want original ratings (no retweets) that have images. Though there are 5000+ tweets in the dataset, not all are dog ratings and some are retweets.
 * Assessing and cleaning the entire dataset completely would require a lot of time, and is not necessary to practice and demonstrate your skills in data wrangling. Therefore, the requirements of this project are only to assess and clean at least 8 quality issues and at least 2 tidiness issues in this dataset.
 * Cleaning includes merging individual pieces of data according to the rules of tidy data. The fact that the rating numerators are greater than the denominators does not need to be cleaned. This unique rating system is a big part of the popularity of WeRateDogs.
 * You do not need to gather the tweets beyond August 1st, 2017. You can, but note that you won't be able to gather the image predictions for these tweets since you don't have access to the algorithm used.


##  Assesment

After gathering each of the above pieces of data, assess them visually and programmatically for quality and tidiness issues. Detect and document at least eight (8) quality issues and two (2) tidiness issues 

In [17]:
twitter_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2517 entries, 0 to 2516
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2517 non-null   object 
 1   in_reply_to_status_id       8 non-null      object 
 2   in_reply_to_user_id         3 non-null      object 
 3   timestamp                   3 non-null      object 
 4   source                      45 non-null     object 
 5   text                        52 non-null     object 
 6   retweeted_status_id         52 non-null     object 
 7   retweeted_status_user_id    52 non-null     object 
 8   retweeted_status_timestamp  52 non-null     object 
 9   expanded_urls               52 non-null     object 
 10  rating_numerator            52 non-null     object 
 11  rating_denominator          52 non-null     object 
 12  name                        9 non-null      object 
 13  doggo                       7 non

In [16]:
twitter_archive.at[0,'tweet_id']

'892420643555336193,,,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Phineas. He\'s a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,,,,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13,10,Phineas,None,None,None,None;;'

### Data tidyness

* .csv file, all data in one column, no consistent deliminator

### Data quality

## Cleaning

### Issues and solutions

### storing the data
Store the clean DataFrame(s) in a CSV file with the main one named `twitter_archive_master.csv`. If additional files exist because multiple tables are required for tidiness, name these files appropriately. Additionally, you may store the cleaned data in a SQLite database (which is to be submitted as well if you do).

# Analysis

. At least three (3) insights and one (1) visualization must be produced.

## Data transformation and augmentation

## Prelimenary analysis

## Statistical Analysis

# Conclusions

Create a 300-600 word written report called `wrangle_report.pdf` or `wrangle_report.html` that briefly describes your wrangling efforts. This is to be framed as an internal document.

Create a 250-word-minimum written report called `act_report.pdf` or `act_report.html` that communicates the insights and displays the visualization(s) produced from your wrangled data. This is to be framed as an external document, like a blog post or magazine article, for example.

Both of these documents can be created in separate Jupyter Notebooks using the Markdown functionality of Jupyter Notebooks, then downloading those notebooks as PDF files or HTML files (see image below). You might prefer to use a word processor like Google Docs or Microsoft Word, however.